# Courseara Capstone project 

By Jayden Geisler


In [1]:
import pandas as pd
import numpy as np



from geopy.geocoders import Nominatim


import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 

# Download and Clean up the data set 

have found multiple data sets for week 3 i use the wikipidia page  

page: https://open.toronto.ca/dataset/neighbourhoods/

page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

In [21]:
Toronto_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [2]:
Toronto_data

NameError: name 'Toronto_data' is not defined

## Get ride of District not assigned
##### clean up data set, remove not assigned district and change the name of not assigned neighbourhoods to the same as the district

In [0]:
index = 0
for i in Toronto_data['Borough']:
    if i == 'Not assigned':
        Toronto_data.drop([index],inplace = True)
    index += 1

In [0]:
Toronto_data.reset_index(drop = True, inplace = True)
Toronto_data

In [0]:
index = 0
for i in Toronto_data['Neighbourhood']:
    if i == 'Not assigned':
        Toronto_data.at[index, 'Neighbourhood'] = Toronto_data['Borough'][index]
    index += 1

In [0]:
Toronto_data

### combine same postal code Neighbourhoods

### Using geolocator geopy. only can use one call per second so have to add a one second timer to loop 

#### does not work with partial postal codes so will have to use Neighbourhood names

In [0]:
from geopy.geocoders import Nominatim
import time 

longit = np.zeros(len(Toronto_data['Neighbourhood']))
latit = np.zeros(len(Toronto_data['Neighbourhood']))

index = 0
for i in Toronto_data['Neighbourhood']:
    geolocator = Nominatim(user_agent="Toronto2")
    location = geolocator.geocode(i + ",Toronto,Ontario,Canada")
    time.sleep(1)
    if location != None:
        print((location.latitude, location.longitude))
        longit[index] = location.longitude
        latit[index] = location.latitude
    index += 1
        


In [0]:
Toronto_data['Latitude'] = latit
Toronto_data['Longitude'] = longit
Toronto_data

#### To get the average location of a postal code average out the Latitude and Longitude of all neighbouthoods in the postal code

In [0]:
longlatave = Toronto_data.groupby(['Postcode', 'Borough'])['Latitude','Longitude'].mean().reset_index()
longlatave

In [0]:
Toronto_data2 = Toronto_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
Toronto_data2.columns = ['Postcode', 'Borough', 'Neighbourhood']
Toronto_data2

In [0]:
Toronto_data2['Latitude'] = longlatave['Latitude']
Toronto_data2['Longitude'] = longlatave['Longitude']
Toronto_data2

# This is the end of Week 3 exercises 
## getting the latitude and longitude form geopy are not the exat same as the given data set 

#### need to make Totonto_data the same as previouse neighbourhood data

In [22]:
url = 'C:\Projects\Coursera_Capstone/Neighbourhoods.csv'
df = pd.read_csv(url)
df.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,2661,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),NaN,NaN,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
1,2662,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),NaN,NaN,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"{u'type': u'Polygon', u'coordinates': (((-79.4..."
2,2663,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),NaN,NaN,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"{u'type': u'Polygon', u'coordinates': (((-79.3..."
3,2664,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),NaN,NaN,-79.488883,43.765736,16491553,2.541821e+07,25632.335240,"{u'type': u'Polygon', u'coordinates': (((-79.5..."
4,2665,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),NaN,NaN,-79.457108,43.714672,16491569,1.156669e+07,13953.408100,"{u'type': u'Polygon', u'coordinates': (((-79.4..."


In [23]:
#make new smaller data set for just name long and lat
column_names = ['Neighborhood','Latitude','Longitude']

neighborhoods = pd.DataFrame(columns = column_names)

In [24]:
neighborhoods['Neighborhood'] = df['AREA_NAME']
neighborhoods['Latitude'] = df['LATITUDE']
neighborhoods['Longitude'] = df['LONGITUDE']


In [25]:
neighborhoods

,Neighborhood,Latitude,Longitude
0,Wychwood (94),43.676919,-79.425515
1,Yonge-Eglinton (100),43.704689,-79.403590
2,Yonge-St.Clair (97),43.687859,-79.397871
3,York University Heights (27),43.765736,-79.488883
4,Yorkdale-Glen Park (31),43.714672,-79.457108
...,...,...,...
135,Kennedy Park (124),43.725556,-79.260382
136,Kensington-Chinatown (78),43.653554,-79.397240
137,Kingsview Village-The Westway (6),43.698993,-79.547863
138,Kingsway South (15),43.653520,-79.510577


#### Remove numbers form the end of the neighborhood names 

In [26]:
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace('\d+', '').str.replace('(', '').str.replace(')','')
neighborhoods


,Neighborhood,Latitude,Longitude
0,Wychwood,43.676919,-79.425515
1,Yonge-Eglinton,43.704689,-79.403590
2,Yonge-St.Clair,43.687859,-79.397871
3,York University Heights,43.765736,-79.488883
4,Yorkdale-Glen Park,43.714672,-79.457108
...,...,...,...
135,Kennedy Park,43.725556,-79.260382
136,Kensington-Chinatown,43.653554,-79.397240
137,Kingsview Village-The Westway,43.698993,-79.547863
138,Kingsway South,43.653520,-79.510577


In [27]:
#neighborhoods.index[['Newtonbrook West ']]
#neighborhoods.drop([0])

# Mapping the Neighborhood with folium

In [28]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [29]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get 4square Credentials

In [31]:
#foursquare keys 
CLIENT_ID = '1J22M2N15T0T1SX1RID4X3ARVEUE0SLBCTCJOS3L1M24RVVW'
CLIENT_SECRET = 'VJAOZHIQOETDQJ0NTU1BOFOVRWUBWFSDHXFDQUCKZWFK4L1X' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1J22M2N15T0T1SX1RID4X3ARVEUE0SLBCTCJOS3L1M24RVVW
CLIENT_SECRET:VJAOZHIQOETDQJ0NTU1BOFOVRWUBWFSDHXFDQUCKZWFK4L1X


In [32]:
LIMIT = 100

radius = 500 

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

NameError: name 'neighborhood_latitude' is not defined

In [33]:
results = requests.get(url).json()

InvalidSchema: No connection adapters were found for 'C:\Projects\Coursera_Capstone/Neighbourhoods.csv'

In [67]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

NameError: name 'results' is not defined

## Exploriong Neighborhoods 
### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    CLIENT_ID = ''
    CLIENT_SECRET = '' 
    VERSION = '20180605'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

In [36]:
#run function on all neighborhoods and make a new dataframe

Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wychwood 


KeyError: 'groups'

In [71]:
# show shape and size of new data frame 
print(Toronto_venues.shape)
Toronto_venues

(2123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wychwood,43.676919,-79.425515,Wychwood Barns Farmers' Market,43.680010,-79.423849,Farmers Market
1,Wychwood,43.676919,-79.425515,Wychwood Barns,43.680028,-79.423810,Event Space
2,Wychwood,43.676919,-79.425515,Hillcrest Park,43.676012,-79.424787,Park
3,Wychwood,43.676919,-79.425515,Bob Coffee Bar,43.675376,-79.423268,Coffee Shop
4,Yonge-Eglinton,43.704689,-79.403590,North Toronto Memorial Community Centre,43.706098,-79.404337,Gym
...,...,...,...,...,...,...,...
2118,L'Amoreaux,43.795716,-79.314084,Black's Photography,43.797292,-79.319332,Camera Store
2119,L'Amoreaux,43.795716,-79.314084,Fit4Less,43.798394,-79.318453,Gym
2120,L'Amoreaux,43.795716,-79.314084,Dollarama,43.798394,-79.318453,Discount Store
2121,L'Amoreaux,43.795716,-79.314084,Olympian Swimming,43.791575,-79.313204,Gym Pool


### 2 rows do not make it to the venue data frame will need to clean this up 

In [72]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,18,18,18,18,18,18
Agincourt South-Malvern West,23,23,23,23,23,23
Alderwood,6,6,6,6,6,6
Annex,26,26,26,26,26,26
Banbury-Don Mills,24,24,24,24,24,24
...,...,...,...,...,...,...
Wychwood,4,4,4,4,4,4
Yonge-Eglinton,39,39,39,39,39,39
Yonge-St.Clair,51,51,51,51,51,51


In [73]:
neighborhoods

,Neighborhood,Latitude,Longitude
0,Wychwood,43.676919,-79.425515
1,Yonge-Eglinton,43.704689,-79.403590
2,Yonge-St.Clair,43.687859,-79.397871
3,York University Heights,43.765736,-79.488883
4,Yorkdale-Glen Park,43.714672,-79.457108
...,...,...,...
135,Kennedy Park,43.725556,-79.260382
136,Kensington-Chinatown,43.653554,-79.397240
137,Kingsview Village-The Westway,43.698993,-79.547863
138,Kingsway South,43.653520,-79.510577


#### Some neighborhoods get taken off because no venues found so must remove those form the neighborhoods dataframe 

In [20]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 289 uniques categories.


# Analyze Each Neighborhood

In [21]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

#get a list of columns
cols = list(Toronto_onehot)

# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))

# use ix to reorder
Toronto_onehot = Toronto_onehot.loc[:, cols]

Toronto_onehot

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Wychwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wychwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wychwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wychwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Yonge-Eglinton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,L'Amoreaux,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2119,L'Amoreaux,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2120,L'Amoreaux,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2121,L'Amoreaux,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and take the mean of the frequency of occurrence of each catagory 

In [41]:
nei_loc = Toronto_onehot.columns.get_loc('Neighborhood')
Toronto_onehot.drop(Toronto_onehot.columns[nei_loc],axis = 1)
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt North,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.055556,0.000000,0.000000,0.000000,0.0,0.0
1,Agincourt South-Malvern West,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.043478,0.000000,0.0,0.0
2,Alderwood,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,Annex,0.0,0.0,0.038462,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Banbury-Don Mills,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.041667,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Wychwood,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
134,Yonge-Eglinton,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.051282,0.000000,0.000000,0.0,0.0
135,Yonge-St.Clair,0.0,0.0,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,...,0.019608,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.019608,0.0,0.0
136,York University Heights,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


### get rid of rows that did not have venues 

In [119]:
counter = 0
index = 0
#print(neighborhood['Newronbrook West '])
for i in neighborhoods['Neighborhood']:
    counter = 0
    for j in Toronto_grouped['Neighborhood']:
        if i == j:
            print(i,' has found ', j)
            counter = 1
            break
    if counter == 0:    
        print('----------------------------------------------------------------')  
        print(i, ' did not find a match!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('----------------------------------------------------------------')
        neighborhoods.drop([index],inplace= True)
        counter = 0
    index += 1

Wychwood   has found  Wychwood 
Yonge-Eglinton   has found  Yonge-Eglinton 
Yonge-St.Clair   has found  Yonge-St.Clair 
York University Heights   has found  York University Heights 
Yorkdale-Glen Park   has found  Yorkdale-Glen Park 
Lambton Baby Point   has found  Lambton Baby Point 
Lansing-Westgate   has found  Lansing-Westgate 
Lawrence Park North   has found  Lawrence Park North 
Lawrence Park South   has found  Lawrence Park South 
Leaside-Bennington   has found  Leaside-Bennington 
Little Portugal   has found  Little Portugal 
Long Branch   has found  Long Branch 
Malvern   has found  Malvern 
Maple Leaf   has found  Maple Leaf 
Markland Wood   has found  Markland Wood 
Milliken   has found  Milliken 
Mimico includes Humber Bay Shores   has found  Mimico includes Humber Bay Shores 
Morningside   has found  Morningside 
Moss Park   has found  Moss Park 
Mount Dennis   has found  Mount Dennis 
Mount Olive-Silverstone-Jamestown   has found  Mount Olive-Silverstone-Jamestown 
Mount 

In [120]:
neighborhoods

,Neighborhood,Latitude,Longitude
0,Wychwood,43.676919,-79.425515
1,Yonge-Eglinton,43.704689,-79.403590
2,Yonge-St.Clair,43.687859,-79.397871
3,York University Heights,43.765736,-79.488883
4,Yorkdale-Glen Park,43.714672,-79.457108
...,...,...,...
135,Kennedy Park,43.725556,-79.260382
136,Kensington-Chinatown,43.653554,-79.397240
137,Kingsview Village-The Westway,43.698993,-79.547863
138,Kingsway South,43.653520,-79.510577


show each neighborhood with top 5 venues 

### top venues in each neighborhood put into a dataframe 

In [122]:
    def return_most_common_venues(row, num_top_venues):
        row_categories = row.iloc[1:]
        row_categories_sorted = row_categories.sort_values(ascending=False)
    
        return row_categories_sorted.index.values[0:num_top_venues]

In [123]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Chinese Restaurant,Pizza Place,Discount Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Beer Store,Sandwich Place,Bakery,Japanese Restaurant
1,Agincourt South-Malvern West,Chinese Restaurant,Shopping Mall,Pool Hall,BBQ Joint,Mediterranean Restaurant,Malay Restaurant,Filipino Restaurant,Cantonese Restaurant,Seafood Restaurant,Noodle House
2,Alderwood,Pizza Place,Dance Studio,Convenience Store,Pharmacy,Coffee Shop,Zoo Exhibit,Film Studio,Farm,Farmers Market,Fast Food Restaurant
3,Annex,Sandwich Place,Coffee Shop,Café,Park,Pub,Indian Restaurant,Burger Joint,Pharmacy,French Restaurant,Shoe Repair
4,Banbury-Don Mills,Shoe Store,Coffee Shop,Gourmet Shop,Cosmetics Shop,Pizza Place,Clothing Store,Chocolate Shop,Sandwich Place,Cantonese Restaurant,Furniture / Home Store
...,...,...,...,...,...,...,...,...,...,...,...
133,Wychwood,Event Space,Park,Farmers Market,Coffee Shop,Filipino Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant,Field,Zoo Exhibit
134,Yonge-Eglinton,Coffee Shop,Fast Food Restaurant,Women's Store,Gym,Burger Joint,Steakhouse,Gas Station,Movie Theater,Breakfast Spot,Supermarket
135,Yonge-St.Clair,Coffee Shop,Italian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bagel Shop,Restaurant,Café,Gym,Grocery Store
136,York University Heights,Furniture / Home Store,Japanese Restaurant,Fast Food Restaurant,Miscellaneous Shop,Bar,Massage Studio,Bank,Coffee Shop,Caribbean Restaurant,Film Studio


# Cluster Neighborhoods

#### cluster neighborhood into 5 clusters 

In [124]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
type(kmeans.labels_[1])

numpy.int32

### make a mega data frame that has all clustering and top 10 venues 

In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(loc = 0, column = 'Cluster Labels', value = kmeans.labels_)

Toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood,43.676919,-79.425515,4,Event Space,Park,Farmers Market,Coffee Shop,Filipino Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant,Field,Zoo Exhibit
1,Yonge-Eglinton,43.704689,-79.403590,4,Coffee Shop,Fast Food Restaurant,Women's Store,Gym,Burger Joint,Steakhouse,Gas Station,Movie Theater,Breakfast Spot,Supermarket
2,Yonge-St.Clair,43.687859,-79.397871,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bagel Shop,Restaurant,Café,Gym,Grocery Store
3,York University Heights,43.765736,-79.488883,1,Furniture / Home Store,Japanese Restaurant,Fast Food Restaurant,Miscellaneous Shop,Bar,Massage Studio,Bank,Coffee Shop,Caribbean Restaurant,Film Studio
4,Yorkdale-Glen Park,43.714672,-79.457108,4,Fast Food Restaurant,Restaurant,Coffee Shop,Bookstore,Greek Restaurant,Grocery Store,Gym,Bank,Seafood Restaurant,Bowling Alley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Kennedy Park,43.725556,-79.260382,4,Discount Store,Convenience Store,Playground,Coffee Shop,Zoo Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
136,Kensington-Chinatown,43.653554,-79.397240,4,Café,Bar,Chinese Restaurant,Mexican Restaurant,Dumpling Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Cocktail Bar,Farmers Market
137,Kingsview Village-The Westway,43.698993,-79.547863,4,Coffee Shop,Bus Line,Grocery Store,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
138,Kingsway South,43.653520,-79.510577,4,Pool,Lounge,Zoo Exhibit,Film Studio,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


# Create Map

In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examin clusters

In [130]:
#cluster 1 
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,43.657420,Park,Zoo Exhibit,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
6,43.754271,Soccer Field,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio
15,43.820691,Park,Zoo Exhibit,Field,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
24,43.791536,Park,Piano Bar,Café,Film Studio,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
27,43.671995,Café,Pool,Dog Run,Zoo Exhibit,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
34,43.755033,Food & Drink Shop,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio
43,43.682820,Playground,Park,Candy Store,Zoo Exhibit,Field,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
54,43.694998,Theater,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio
62,43.767490,Construction & Landscaping,Gym / Fitness Center,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
64,43.778813,American Restaurant,Home Service,Park,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [131]:
# cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,43.765736,Japanese Restaurant,Fast Food Restaurant,Miscellaneous Shop,Bar,Massage Studio,Bank,Coffee Shop,Caribbean Restaurant,Film Studio
16,43.615924,Grocery Store,Train Station,Bakery,Skating Rink,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio
32,43.696781,Pastry Shop,Seafood Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Electronics Store
35,43.717515,Lawyer,Department Store,Bakery,Gift Shop,Skating Rink,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant
38,43.666051,Playground,Zoo Exhibit,Film Studio,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
46,43.711609,Construction & Landscaping,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Zoo Exhibit
71,43.766740,French Restaurant,Bakery,Construction & Landscaping,Fish & Chips Shop,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market
81,43.776361,Construction & Landscaping,Golf Driving Range,Dog Run,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
105,43.759274,Pet Store,Indian Restaurant,Electronics Store,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Flea Market,Fish Market,Flower Shop
125,43.790775,Home Service,Bus Station,Zoo Exhibit,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


In [132]:
#cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,43.770602,Zoo Exhibit,Film Studio,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


In [133]:
#cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,43.683645,Fried Chicken Joint,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio


In [134]:
#cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.676919,Park,Farmers Market,Coffee Shop,Filipino Restaurant,Falafel Restaurant,Farm,Fast Food Restaurant,Field,Zoo Exhibit
1,43.704689,Fast Food Restaurant,Women's Store,Gym,Burger Joint,Steakhouse,Gas Station,Movie Theater,Breakfast Spot,Supermarket
2,43.687859,Italian Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bagel Shop,Restaurant,Café,Gym,Grocery Store
4,43.714672,Restaurant,Coffee Shop,Bookstore,Greek Restaurant,Grocery Store,Gym,Bank,Seafood Restaurant,Bowling Alley
7,43.730060,Bakery,Italian Restaurant,Burger Joint,Asian Restaurant,Sandwich Place,Coffee Shop,Bank,Pub,Shoe Store
...,...,...,...,...,...,...,...,...,...,...
135,43.725556,Convenience Store,Playground,Coffee Shop,Zoo Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
136,43.653554,Bar,Chinese Restaurant,Mexican Restaurant,Dumpling Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Cocktail Bar,Farmers Market
137,43.698993,Bus Line,Grocery Store,Zoo Exhibit,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
138,43.653520,Lounge,Zoo Exhibit,Film Studio,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
